# Hackaton UBU 2025
## Problema de optimización - Supermercados y viviendas
**Autor**: [José Gallardo Caballero](mailto:jgc1031@alu.ubu.es)

<table>
    <tr>
        <td align="center"><a href="https://joseleelportfolio.vercel.app"><img src="https://github.com/Joseleelsuper.png" width="100px;" alt=""/><br /><sub><b>José Gallardo</b></sub></a></td>
    </tr>
</table>

En este ejercicio se nos planteaban dos problemas distintos, pero mi atención fue hacia el problema de los supermercados y viviendas.

El problema consiste en encontrar la solución óptima basándonos en el menor coste y número de supermercados en la zona afectada.
La restrición de esto es que todas y cada una de las viviendas deben de estar dentro de la zona de influyencia de al menos un supermercado.

Para mi solución, he utilizado un algoritmo `greedy`. Por desgracia, este tipo de algoritmos no tienen la certeza de encontrar la solución óptima global, con posibilidad de quedare en una solución local. El algoritmo tiene como objetivo seleccionar ubicaciones para supermercados que maximicen la cobertura de viviendas mientras se minimizan los costes.

### Inicialización:
- Identifica todas las ubicaciones de viviendas en la cuadrícula
- Calcula qué viviendas podría cubrir cada ubicación potencial de supermercado (dentro del radio)

### Bucle de selección de supermercados:
- Selecciona repetidamente la mejor ubicación basada en una "relación beneficio-coste"
- La relación es: número de viviendas recién cubiertas / coste de colocar un supermercado
- Después de seleccionar una ubicación, elimina las viviendas recién cubiertas del conjunto "no cubiertas"
- Continúa hasta que todas las viviendas estén cubiertas o no existan más ubicaciones beneficiosas

### Fase de asignación:
- Asigna cada vivienda a su supermercado más cercano que la cubra



In [9]:
# Librerías necesarias
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import hashlib
import time
import concurrent.futures
import random

In [ ]:
# Ficheros
JSON_ENTRADA = "./Problema/instancia_problema.json"
JSON_SALIDA = "EquipoReturn.json"

In [11]:
class Grid:
    def __init__(self, costs: np.ndarray, homes: np.ndarray, size: tuple, radius: int):
        """Constructor de la clase Grid.
        Representa la cuadrícula del problema de optimización.

        Args:
            costs (np.ndarray): Costos de cada celda en la cuadrícula
            homes (np.ndarray): Número de viviendas en cada celda de la cuadrícula
            size (tuple): Tamaño de la cuadrícula (filas, columnas)
            radius (int): Radio de influencia de cada supermercado
        """
        self.costs = costs
        self.homes = homes
        self.size = size
        self.radius = radius


class SupermarketOptimizer:
    def __init__(self, grid: Grid):
        """Constructor de la clase SupermarketOptimizer.
        Representa el optimizador de supermercados.
        Utiliza un algoritmo de cobertura de conjunto codicioso para resolver el problema.

        Args:
            grid (Grid): La cuadrícula del problema de optimización.
        """
        self.grid = grid

    def solve(self):
        """Resuelve el problema de optimización de supermercados utilizando un algoritmo codicioso.
        El algoritmo selecciona supermercados de manera codiciosa para maximizar la cobertura de viviendas
        """

        N = self.grid.size[0]
        # Coordinaciones de las viviendas
        homes = [tuple(h) for h in np.argwhere(np.greater(self.grid.homes, 0))]
        radius2 = self.grid.radius**2
        # Preparar la cobertura de cada celda
        coverage = {}
        for i in range(N):
            for j in range(N):
                loc = (i, j)
                cov = [h for h in homes if (i - h[0]) ** 2 + (j - h[1]) ** 2 <= radius2]
                if cov:
                    coverage[loc] = cov
        # Seleccionar supermercados
        uncovered = set(homes)
        selected = []
        while uncovered:
            best = None
            best_ratio = -1
            for loc, cov in coverage.items():
                newcov = set(cov) & uncovered
                if newcov:
                    ratio = len(newcov) / float(self.grid.costs[loc])
                    if ratio > best_ratio:
                        best_ratio = ratio
                        best = loc
            if best is None:
                break
            selected.append(best)
            uncovered -= set(coverage[best])
        # Guardar la solución
        self.supermarkets = selected
        self.total_cost = sum(self.grid.costs[i, j] for i, j in selected)
        # Asignamos viviendas a supermercados
        assignment = {str(loc): [] for loc in selected}
        for h in homes:
            # Rellenamos la lista de viviendas que cubre cada supermercado
            covers = [loc for loc in selected if h in coverage[loc]]
            if not covers:
                continue
            # Y elegimos el más cercano para evitar duplicados en las listas
            nearest = min(
                covers, key=lambda loc: (loc[0] - h[0]) ** 2 + (loc[1] - h[1]) ** 2
            )
            assignment[str(nearest)].append(list(h))
        self.assignment = assignment

    def get_solution(self) -> dict:
        """Devuelve la solución del problema de optimización de supermercados.

        Returns:
            dict: Diccionario con la solución que incluye:
                - supermercados: lista de coordenadas de los supermercados
                - costo_total: costo total de los supermercados
                - n_supermercados: número de supermercados
                - asignacion: diccionario con la asignación de viviendas a supermercados
        """
        return {
            "supermercados": self.supermarkets,
            "costo_total": int(self.total_cost),
            "n_supermercados": len(self.supermarkets),
            "asignacion": self.assignment,
        }

In [12]:
def load_instance(json_path: str) -> Grid:
    """Carga la instancia del problema desde un archivo JSON.
    El archivo JSON debe contener la cuadrícula y la información de las viviendas.

    El formato esperado es el siguiente para los tests:

    ```
    {
        "instancia": {
            "N": <number of rows and columns>,
            "M": <radius of influence>,
            "C": [[<costs>]],
            "viviendas": [[<row>, <column>], ...]
        }
    }
    ```

    O si no para la prueba real:

    ```
    {
        "N": <number of rows and columns>,
        "M": <radius of influence>,
        "C": [[<costs>]],
        "viviendas": [[<row>, <column>], ...]
    }
    ```

    Args:
        json_path (str): Ruta al archivo JSON que contiene la instancia del problema.

    Returns:
        Grid: La cuadrícula del problema de optimización.
    """
    print("Loading data...")

    with open(json_path, "r") as f:
        content = f.read()
    data_loaded = json.loads(content)
    if isinstance(data_loaded, dict) and "instancia" in data_loaded:
        data = data_loaded["instancia"]
    else:
        data = data_loaded

    N = data["N"]
    M = data["M"]
    C = np.array(data["C"])
    homes = np.zeros((N, N), dtype=int)
    for home in data["viviendas"]:
        homes[home[0], home[1]] = 1

    print("Data loaded.")
    return Grid(C, homes, (N, N), M)

In [13]:
def save_solution(solution: dict, json_path: str):
    """Guarda la solución del problema en un archivo JSON.

    Args:
        solution (dict): La solución del problema de optimización.
        json_path (str): Ruta al archivo JSON donde se guardará la solución.
    """
    solucion = {
        "solucion": {
            "supermercados": [
                list(map(int, s)) for s in solution.get("supermercados", [])
            ],
            "costo_total": int(solution.get("costo_total", 0)),
            "n_supermercados": int(solution.get("n_supermercados", 0)),
            "asignacion": {
                str(list(map(int, eval(k)))): [list(map(int, v_)) for v_ in v]
                for k, v in solution.get("asignacion", {}).items()
            },
        }
    }
    with open(json_path, "w") as f:
        json.dump(solucion, f)

In [14]:
def plot_problem(grid, output_path="problema.png"):
    """Creamos la imagen del problema.
    Esta imagen muestra las viviendas.

    Args:
        grid (Grid): La cuadrícula del problema de optimización.
        output_path (str): Ruta al archivo de salida donde se guardará la imagen.
    """
    N = grid.size[0]
    _, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(-1, N)
    ax.set_ylim(-1, N)
    ax.set_aspect("equal")
    ax.set_xticks([])
    ax.set_yticks([])

    # Mostrar viviendas
    viviendas = np.argwhere(grid.homes > 0)
    if len(viviendas) > 0:
        ax.scatter(viviendas[:, 1], viviendas[:, 0], c="blue", s=10, label="Viviendas")

    ax.set_title("Problema de Optimización")
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()


def plot_solution(grid, solution, output_path="solucion.png"):
    """Creamos la imagen de la solución.
    Esta imagen muestra las viviendas y los supermercados seleccionados.
    También muestra el radio de influencia de cada supermercado.

    Args:
        grid (Grid): La cuadrícula del problema de optimización.
        solution (dict): La solución del problema de optimización.
        output_path (str): Ruta al archivo de salida donde se guardará la imagen.
    """
    N = grid.size[0]
    _, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(-1, N)
    ax.set_ylim(-1, N)
    ax.set_aspect("equal")
    ax.set_xticks([])
    ax.set_yticks([])

    # Mostrar viviendas
    viviendas = np.argwhere(grid.homes > 0)
    if len(viviendas) > 0:
        ax.scatter(viviendas[:, 1], viviendas[:, 0], c="blue", s=10, label="Viviendas")

    # Mostrar supermercados y su radio
    for supermercado in solution["supermercados"]:
        y, x = supermercado
        circle = patches.Circle(
            (x, y), grid.radius, fill=False, edgecolor="red", linestyle="--", alpha=0.3
        )
        ax.add_patch(circle)
        ax.scatter(x, y, c="red", s=40, marker="*", label="Supermercado")  # type: ignore

    ax.set_title("Solución de Optimización")
    # Evitar duplicados en la leyenda
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys())
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

In [15]:
def save_instance_hash(input_path, folder):
    """Calcula y guarda el hash de la instancia.

    Args:
        input_path (str): Ruta del archivo de entrada.
        folder (str): Carpeta donde se guardará el hash.

    Returns:
        str: El hash calculado.
    """
    # calcular hash de la instancia
    with open(input_path, "rb") as f:
        data = f.read()
    h = hashlib.sha256(data).hexdigest()
    # guardar hash
    hash_path = os.path.join(folder, "hash.txt")
    with open(hash_path, "w") as hf:
        hf.write(h)
    return h

In [16]:
def main():
    input_path = JSON_ENTRADA
    output_json = JSON_SALIDA

    # carpeta con nombre de la salida
    folder = os.path.splitext(os.path.basename(output_json))[0]
    # comprobar si el hash ya existe en alguna carpeta para evitar análisis duplicado
    # calcular hash de la instancia sin guardarlo aún
    with open(input_path, "rb") as f:
        new_hash = hashlib.sha256(f.read()).hexdigest()
    for d in os.listdir("."):
        if os.path.isdir(d):
            hash_file = os.path.join(d, "hash.txt")
            if os.path.exists(hash_file):
                with open(hash_file, "r") as hf:
                    if hf.read().strip() == new_hash:
                        print(f"La instancia ya fue analizada en la carpeta {d}.")
                # Buscar el archivo json de solución en la carpeta
                for file in os.listdir(d):
                    if file.endswith(".json") and file != os.path.basename(input_path):
                        json_path = os.path.join(d, file)
                        with open(json_path, "r") as jf:
                            sol_data = json.load(jf)
                            if "solucion" in sol_data:
                                cost = sol_data["solucion"]["costo_total"]
                                n_supermarkets = sol_data["solucion"]["n_supermercados"]
                                print(
                                    f"Costo total = {cost}\nNúmero de supermercados = {n_supermarkets}"
                                )
                        break
                return

    os.makedirs(folder, exist_ok=True)

    # calcular y guardar hash de la instancia
    save_instance_hash(input_path, folder)

    # cargar instancia
    grid = load_instance(input_path)
    # rutas de salida dentro de la carpeta
    json_out = os.path.join(folder, os.path.basename(output_json))
    problem_image = os.path.join(folder, f"{folder}_problem.png")
    solution_image = os.path.join(folder, f"{folder}_solution.png")
    # generar imagen del problema
    plot_problem(grid, output_path=problem_image)
    print(f"Imagen del problema guardada como {problem_image}")
    # resolver y guardar solución
    start_time = time.time()
    optimizer = SupermarketOptimizer(grid)
    optimizer.solve()
    solution = optimizer.get_solution()
    end_time = time.time()

    save_solution(solution, json_out)
    print(f"Solución guardada en {json_out}")
    # generar imagen de la solución
    plot_solution(grid, solution, output_path=solution_image)
    print(f"Imagen de la solución guardada como {solution_image}")
    print(f"Tiempo de ejecución: {end_time - start_time:.2f} segundos")


if __name__ == "__main__":
    main()

Loading data...
Data loaded.
Imagen del problema guardada como Gallardo_Jose_optimizacion\Gallardo_Jose_optimizacion_problem.png
Solución guardada en Gallardo_Jose_optimizacion\Gallardo_Jose_optimizacion.json
Imagen de la solución guardada como Gallardo_Jose_optimizacion\Gallardo_Jose_optimizacion_solution.png
Tiempo de ejecución: 23.52 segundos


## Referencias
- https://learning.edx.org/course/course-v1:HarvardX+CS50AI+1T2020/home [09/05/2025]